In [25]:
import os
import glob
import yaml
import shutil
from xml.dom import minidom
from ultralytics import YOLO


## Convert xaml to .txt labels


In [ ]:

lut = {"Without Helmet": 0, "With Helmet": 1}

def get_text(tag, parent):
    return parent.getElementsByTagName(tag)[0].firstChild.data

def convert_coordinates(size, box):
    dw, dh = 1.0 / size[0], 1.0 / size[1]
    x = (box[0] + box[1]) / 2.0 * dw
    y = (box[2] + box[3]) / 2.0 * dh
    w = (box[1] - box[0]) * dw
    h = (box[3] - box[2]) * dh
    return x, y, w, h

def convert_xml2yolo(lut, input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for fname in glob.glob(os.path.join(input_folder, "*.xml")):
        xmldoc = minidom.parse(fname)
        size = xmldoc.getElementsByTagName("size")[0]
        width = int(get_text("width", size))
        height = int(get_text("height", size))
        fname_out = os.path.join(output_folder, os.path.basename(fname).replace(".xml", ".txt"))

        with open(fname_out, "w") as f:
            for item in xmldoc.getElementsByTagName("object"):
                class_name = get_text("name", item)
                if class_name not in lut:
                    print(f" Unknown label '{class_name}' in {fname}. Skipping.")
                    continue
                bbox = item.getElementsByTagName("bndbox")[0]
                xmin = float(get_text("xmin", bbox))
                xmax = float(get_text("xmax", bbox))
                ymin = float(get_text("ymin", bbox))
                ymax = float(get_text("ymax", bbox))
                bb = convert_coordinates((width, height), (xmin, xmax, ymin, ymax))
                f.write(f"{lut[class_name]} " + " ".join(f"{x:.6f}" for x in bb) + "\n")

        print(f" Converted: {fname_out}")

# Run the converter
convert_xml2yolo(lut, "annotations", "labels")



## Train Test And Validation Split

In [ ]:

import os
import random
import shutil

# Paths
image_dir = "images"
label_dir = "labels"

train_img_dir = "images/train"
val_img_dir = "images/val"
test_img_dir = "images/test"

train_lbl_dir = "labels/train"
val_lbl_dir = "labels/val"
test_lbl_dir = "labels/test"

# Create folders
for d in [train_img_dir, val_img_dir, test_img_dir, train_lbl_dir, val_lbl_dir, test_lbl_dir]:
    os.makedirs(d, exist_ok=True)

# Split ratios
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

# Get all image filenames
images = [f for f in os.listdir(image_dir) if f.endswith(".png")]
random.shuffle(images)

n = len(images)
train_files = images[:int(n * train_ratio)]
val_files = images[int(n * train_ratio):int(n * (train_ratio + val_ratio))]
test_files = images[int(n * (train_ratio + val_ratio)):]

def move_files(files, img_dest, lbl_dest):
    for f in files:
        shutil.move(os.path.join(image_dir, f), os.path.join(img_dest, f))
        label_file = f.replace(".png", ".txt")
        shutil.move(os.path.join(label_dir, label_file), os.path.join(lbl_dest, label_file))

move_files(train_files, train_img_dir, train_lbl_dir)
move_files(val_files, val_img_dir, val_lbl_dir)
move_files(test_files, test_img_dir, test_lbl_dir)

print("✅ Dataset successfully split into train/val/test.")



'\nimport os\nimport random\nimport shutil\n\n# Paths\nimage_dir = "images"\nlabel_dir = "labels"\n\ntrain_img_dir = "images/train"\nval_img_dir = "images/val"\ntest_img_dir = "images/test"\n\ntrain_lbl_dir = "labels/train"\nval_lbl_dir = "labels/val"\ntest_lbl_dir = "labels/test"\n\n# Create folders\nfor d in [train_img_dir, val_img_dir, test_img_dir, train_lbl_dir, val_lbl_dir, test_lbl_dir]:\n    os.makedirs(d, exist_ok=True)\n\n# Split ratios\ntrain_ratio = 0.7\nval_ratio = 0.2\ntest_ratio = 0.1\n\n# Get all image filenames\nimages = [f for f in os.listdir(image_dir) if f.endswith(".png")]\nrandom.shuffle(images)\n\nn = len(images)\ntrain_files = images[:int(n * train_ratio)]\nval_files = images[int(n * train_ratio):int(n * (train_ratio + val_ratio))]\ntest_files = images[int(n * (train_ratio + val_ratio)):]\n\ndef move_files(files, img_dest, lbl_dest):\n    for f in files:\n        shutil.move(os.path.join(image_dir, f), os.path.join(img_dest, f))\n        label_file = f.replace("

## yaml file config

In [27]:

project_root = os.getcwd()
images_path = os.path.join(project_root, "images")

data = {
    "train": os.path.join(images_path, "train"),
    "val": os.path.join(images_path, "val"),
    "test": os.path.join(images_path, "test"),
    "nc": 2,
    "names": ["Without Helmet", "With Helmet"]
}

yaml_path = os.path.join(project_root, "dataset.yaml")
with open(yaml_path, "w") as f:
    yaml.dump(data, f)

print(f"✅ Fixed dataset.yaml with absolute paths at {yaml_path}")

✅ Fixed dataset.yaml with absolute paths at d:\WT LAB\Mini Project\dataset.yaml


## Import YOLOv8

In [ ]:
model = YOLO("yolov8n.pt")  

results = model.train(data="dataset.yaml", epochs=20)
print("Training complete!")


## Test the model

In [21]:
import matplotlib.pyplot as plt

# Test the model on a sample image
results = model("d:\\WT LAB\\Mini Project\\images\\test\\BikesHelmets19.png")  # Provide the path to your image

# Show the results (detection and annotations)
annotated_image = results[0].plot()

# Display the annotated image using matplotlib
plt.imshow(annotated_image)
plt.axis('off')  # Hide axes for a cleaner view
plt.show()



image 1/1 d:\WT LAB\Mini Project\images\test\BikesHelmets19.png: 480x640 2 persons, 2 bicycles, 1 bottle, 228.2ms
Speed: 4.6ms preprocess, 228.2ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)


<Figure size 640x480 with 1 Axes>

## Training


In [24]:
results = model.train(data="dataset.yaml", epochs=10)

print("Training complete!")

Ultralytics 8.3.109  Python-3.11.0 torch-2.6.0+cpu CPU (AMD Ryzen 7 5800HS with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=dataset.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width

train: Scanning D:\WT LAB\Mini Project\labels\train.cache... 534 images, 2 backgrounds, 11 corrupt: 100%|██████████| 534/534 [00:00<?, ?it/s]

train: WARNING  D:\WT LAB\Mini Project\images\train\BikesHelmets103.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [408.72375  81.5019   71.0375   83.0038 ]
train: WARNING  D:\WT LAB\Mini Project\images\train\BikesHelmets140.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [547.685  84.5   132.165 151.   ]
train: WARNING  D:\WT LAB\Mini Project\images\train\BikesHelmets205.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [263.5  43.   85.   74. ]
train: WARNING  D:\WT LAB\Mini Project\images\train\BikesHelmets343.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [682.27 150.5  118.22 107.  ]
train: WARNING  D:\WT LAB\Mini Project\images\train\BikesHelmets441.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [310.5  41.5  61.   63. ]
train: WARNING  D:\WT LAB\Mini Project\images\train\BikesHelmets444.png: ignoring corrupt image/label: non-


val: Scanning D:\WT LAB\Mini Project\labels\val.cache... 153 images, 0 backgrounds, 2 corrupt: 100%|██████████| 153/153 [00:00<?, ?it/s]

val: WARNING  D:\WT LAB\Mini Project\images\val\BikesHelmets706.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [379.5  82.5  61.   65. ]
val: WARNING  D:\WT LAB\Mini Project\images\val\BikesHelmets764.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [219.  91.  80.  82.]
Plotting labels to runs\detect\train3\labels.jpg... 


...

      10/10         0G       1.27      1.168      1.124         18        640: 100%|██████████| 33/33 [03:20<00:00,  6.08s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:17<00:00,  3.51s/it]

                   all        151        285      0.759      0.694      0.769      0.463



10 epochs completed in 0.623 hours.
Optimizer stripped from runs\detect\train3\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train3\weights\best.pt, 6.2MB

Validating runs\detect\train3\weights\best.pt...
Ultralytics 8.3.109  Python-3.11.0 torch-2.6.0+cpu CPU (AMD Ryzen 7 5800HS with Radeon Graphics)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:16<00:00,  3.28s/it]


                   all        151        285      0.746      0.707      0.776      0.466
        Without Helmet         65        124      0.699      0.601      0.699      0.366
           With Helmet         97        161      0.792      0.814      0.854      0.567
Speed: 3.4ms preprocess, 76.3ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to runs\detect\train3
Training complete!


In [28]:
model.save("models/trained_model_e10.pt")

In [36]:

model = YOLO("runs/detect/train3/weights/best.pt")

# Evaluate on your test set
metrics = model.val(data="dataset.yaml", split='test')  # if test set is defined in dataset.yaml
 

Ultralytics 8.3.109  Python-3.11.0 torch-2.6.0+cpu CPU (AMD Ryzen 7 5800HS with Radeon Graphics)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning D:\WT LAB\Mini Project\labels\test.cache... 77 images, 1 backgrounds, 2 corrupt: 100%|██████████| 77/77 [00:00<?, ?it/s]

val: WARNING  D:\WT LAB\Mini Project\images\test\BikesHelmets279.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [194.5  56.  103.  100. ]
val: WARNING  D:\WT LAB\Mini Project\images\test\BikesHelmets326.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [157.   56.5  86.   97. ]



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:07<00:00,  1.52s/it]


                   all         75        136       0.74      0.736      0.775      0.486
        Without Helmet         21         31      0.623      0.677      0.645      0.398
           With Helmet         57        105      0.856      0.794      0.905      0.573
Speed: 1.4ms preprocess, 70.5ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to runs\detect\val5


In [42]:

print(f"mAP@0.5: {metrics.box.map50:.4f}")
print(metrics.box)

mAP@0.5: 0.7749
ultralytics.utils.metrics.Metric object with attributes:

all_ap: array([[    0.64498,     0.61066,     0.59308,     0.56416,     0.52046,     0.44604,     0.36346,     0.16042,     0.07659,   0.0025443],
       [    0.90476,     0.90349,     0.87764,     0.81473,     0.77072,     0.67276,     0.49738,     0.24385,    0.043661,           0]])
ap: array([    0.39824,      0.5729])
ap50: array([    0.64498,     0.90476])
ap_class_index: array([0, 1])
curves: []
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.03

In [43]:


model = YOLO("runs/detect/train3/weights/last.pt")
model.train(data="dataset.yaml", epochs=10)  # Will train for 10 more epochs (total = 10 + 10)


Ultralytics 8.3.109  Python-3.11.0 torch-2.6.0+cpu CPU (AMD Ryzen 7 5800HS with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=runs/detect/train3/weights/last.pt, data=dataset.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, sho

train: Scanning D:\WT LAB\Mini Project\labels\train.cache... 534 images, 2 backgrounds, 11 corrupt: 100%|██████████| 534/534 [00:00<?, ?it/s]

train: WARNING  D:\WT LAB\Mini Project\images\train\BikesHelmets103.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [408.72375  81.5019   71.0375   83.0038 ]
train: WARNING  D:\WT LAB\Mini Project\images\train\BikesHelmets140.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [547.685  84.5   132.165 151.   ]
train: WARNING  D:\WT LAB\Mini Project\images\train\BikesHelmets205.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [263.5  43.   85.   74. ]
train: WARNING  D:\WT LAB\Mini Project\images\train\BikesHelmets343.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [682.27 150.5  118.22 107.  ]
train: WARNING  D:\WT LAB\Mini Project\images\train\BikesHelmets441.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [310.5  41.5  61.   63. ]
train: WARNING  D:\WT LAB\Mini Project\images\train\BikesHelmets444.png: ignoring corrupt image/label: non-


val: Scanning D:\WT LAB\Mini Project\labels\val.cache... 153 images, 0 backgrounds, 2 corrupt: 100%|██████████| 153/153 [00:00<?, ?it/s]

val: WARNING  D:\WT LAB\Mini Project\images\val\BikesHelmets706.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [379.5  82.5  61.   65. ]
val: WARNING  D:\WT LAB\Mini Project\images\val\BikesHelmets764.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [219.  91.  80.  82.]
Plotting labels to runs\detect\train4\labels.jpg... 


...

                   all        151        285      0.753      0.718      0.788      0.473



10 epochs completed in 0.615 hours.
Optimizer stripped from runs\detect\train4\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train4\weights\best.pt, 6.2MB

Validating runs\detect\train4\weights\best.pt...
Ultralytics 8.3.109  Python-3.11.0 torch-2.6.0+cpu CPU (AMD Ryzen 7 5800HS with Radeon Graphics)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:15<00:00,  3.17s/it]


                   all        151        285      0.754      0.718      0.788      0.472
        Without Helmet         65        124      0.781      0.597      0.711      0.377
           With Helmet         97        161      0.727      0.839      0.865      0.568
Speed: 3.0ms preprocess, 74.6ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs\detect\train4


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001AA71B2A690>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0

In [44]:

model = YOLO("runs/detect/train4/weights/last.pt")
model.save("models/trained_model_e20.pt")

In [60]:
model.names  # This gives you a dictionary {0: 'person', 1: 'bicycle', ...}


{0: 'Helmet', 1: 'No Helmet'}

In [62]:

model = YOLO("yolov8n.pt")
model.names

{0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mou

## Triple Riding


In [65]:
def iou(box1, box2):
    # Get coordinates
    x1_min, y1_min, x1_max, y1_max = box1.xyxy[0]
    x2_min, y2_min, x2_max, y2_max = box2.xyxy[0]

    # Calculate intersection
    inter_xmin = max(x1_min, x2_min)
    inter_ymin = max(y1_min, y2_min)
    inter_xmax = min(x1_max, x2_max)
    inter_ymax = min(y1_max, y2_max)

    inter_area = max(0, inter_xmax - inter_xmin) * max(0, inter_ymax - inter_ymin)

    # Calculate union
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    union_area = box1_area + box2_area - inter_area

    # IoU
    return inter_area / union_area if union_area > 0 else 0


In [66]:

model = YOLO("yolov8n.pt")
results = model("D:\\WT LAB\Mini Project\\images\\test\\BikesHelmets30.png")[0]
boxes = results.boxes
persons = [b for b in boxes if b.cls == 0]
bikes = [b for b in boxes if b.cls == 1]

for bike in bikes:
    count = 0
    for person in persons:
        if iou(person, bike) > 0.3:  # or use center-in-bbox
            count += 1
    if count >= 3:
        print("Triple riding detected!")



image 1/1 D:\WT LAB\Mini Project\images\test\BikesHelmets30.png: 640x448 4 persons, 2 bicycles, 2 cars, 101.9ms
Speed: 12.2ms preprocess, 101.9ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 448)
Triple riding detected!


In [1]:
model = YOLO("runs/detect/train4/weights/best.pt")  # path to your trained weights

# Run inference on a video file

# Run prediction on a video and save the output
model.predict(
    source="D:\\WT LAB\\Mini Project\\3767432883-preview.mp4",   # input video path
    conf=0.25,                   # confidence threshold
    save=True,                   # save output video
    imgsz=369                    # image size (match your training size)
)



NameError: name 'YOLO' is not defined

In [2]:
model = YOLO("runs/detect/train4/weights/best.pt")  # path to your trained weights

# Run prediction on a video and save the output
model.predict(
    source="D:\\WT LAB\\Mini Project\\Revolutionizing Road Safety with AI-Powered Helmet Detection _ Smart Cities by Kundan kumar.mp4",   # input video path
    conf=0.45,                   # confidence threshold
    save=True,                   # save output video
    imgsz=640                    # image size (match your training size)
)


NameError: name 'YOLO' is not defined

In [3]:
model = YOLO("helmet.pt")  # path to your trained weights

# Run prediction on a video and save the output
model.predict(
    source="D:\\WT LAB\\Mini Project\\Revolutionizing Road Safety with AI-Powered Helmet Detection _ Smart Cities by Kundan kumar.mp4",   # input video path
    conf=0.45,                   # confidence threshold
    save=True,                   # save output video
    imgsz=640                    # image size (match your training size)
)


NameError: name 'YOLO' is not defined

## Testing

In [ ]:
# Create folder to save results
os.makedirs("results", exist_ok=True)

# Get .png test images
test_images = glob.glob("images/test*.png")

# Run inference on first 3 test images
for img_path in test_images[:3]:
    results = model(img_path)

    for result in results:
        result.show()
        result.save(filename=os.path.join("results", f"result_{os.path.basename(img_path)}"))

    print(f"✅ Processed {img_path}")


In [5]:
model = YOLO("runs/detect/train6/weights/best.pt")
model.predict(
    source="D:\\WT LAB\\Mini Project\\Revolutionizing Road Safety with AI-Powered Helmet Detection _ Smart Cities by Kundan kumar.mp4",   # input video path
    conf=0.45,                   # confidence threshold
    save=True,                   # save output video
    imgsz=369                    # image size (match your training size)
)

NameError: name 'YOLO' is not defined